In [38]:
job_description = """
 Opportunities with Episource, part of the Optum family of businesses. Join a premier provider of risk adjustment services, software and solutions that’s fueling innovation in the health care industry. Start a rewarding career where your work will empower health plans and medical groups with comprehensive end-to-end solutions designed to navigate health care efficiently. Our culture is rooted in innovation, encouraging our team to stay curious and engaged. By joining us, you become part of a global, remote/hybrid-friendly team dedicated to bridging health care gaps with a solid sense of social responsibility. At Episource, we are enriching lives, including those of our team members through Caring. Connecting. Growing together. 

As a Data Analyst, you will play a crucial role in transforming raw data into actionable insights that drive business decisions and strategies. Your responsibilities will include data collection, analysis, interpretation, and transformation, helping stakeholders understand complex datasets to make informed choices and optimize performance

The data analyst will execute data pipelines, analyze transformed data to validate its completeness and gather insight to impact business decisions. Additionally, the data analysis will handle ad hoc analysis request-related data surrounding medical Record retrieval to assist in determining the location of medical charts.

You’ll enjoy the flexibility to work remotely * from anywhere within the U.S. as you take on some tough challenges.

Primary Responsibilities

    Collect, clean, and preprocess data from various sources to ensure accuracy and consistency
    Identify and address data quality issues, outliers, and missing values
    Perform exploratory data analysis to uncover trends, patterns, and relationships within datasets
    Oversee ingestion pipelines and troubleshoots errors related to data. Resolve errors using approved data cleaning techniques
    Translate complex data analysis into clear and actionable insights for stakeholders
    Collaborate with cross-functional teams to identify opportunities for process optimization and strategic decision-making
    Build analytics tools that utilize the data pipeline to provide actionable insights into operational efficiency, and other critical business performance metrics
    Develops and maintains solid relationships with all internal and external stakeholders
    Document analysis methodologies, assumptions, and findings for future reference and replication
    Requires an individual to maintain the ability to work in an environment with PHI / PII data 

You’ll be rewarded and recognized for your performance in an environment that will challenge you and give you clear direction on what it takes to succeed in your role as well as provide development for other roles you may be interested in.

Required Qualifications

    2+ years of experience in data analytics
    2+ years of experience writing intermediate to advanced SQL (joining many tables, SQL functions, CTE's)
    Experience working with Python (Ability to read and understand Python scripts and ability to make adjustments to scripts)
    Experience with data visualization tools such as Tableau 
    Proven knowledge of data analysis and data quality best practices
    Demonstrated excellent verbal and written communication skills, including ability to effectively communicate with internal and external customers
    Proficient in Excel (v-look ups, pivot tables)
    Ability and willingness to work occasional weekends and/or evening work

Preferred Qualifications

    Proven knowledge of healthcare data including member, claims, and provider data
    Proven knowledge of industry standards and practices related to medical records retrieval
    Demonstrated understanding of data pipelines, database structures and ETL practices 
    All employees working remotely will be required to adhere to UnitedHealth Group’s Telecommuter Policy.




"""


In [39]:
import json

with open('../resume.json') as f:
    resume_json = json.load(f)

In [40]:
candidate_exp = resume_json['work experience']

In [41]:
from groq import Groq
import os
import datetime
import subprocess
import time
from dotenv import load_dotenv

In [42]:
load_dotenv()
api_key = os.getenv('GROQ_API_KEY')
client = Groq(api_key=api_key)

In [44]:


def analyze_exp(candidate_profile: dict, job_description: dict, max_trials: int = 5) -> None:
    # Step 1: Define the prompt for optimizing the experience section
    prompt = f"""
    You are an expert resume writer focused on optimizing profiles for ATS (Applicant Tracking System) compliance and job application success. Your task is to enhance the candidate's "Experience" section based on the provided job description:

    1. **Extract Key Responsibilities and Achievements**:
       - From the job description, identify and extract key responsibilities and metrics associated with the ideal candidate's role.

    2. **Optimize the Candidate’s Experience Section**:
       - Carefully review the "Experience" section and rewrite each bullet point to align with the job description and ATS compatibility.
       - Follow the STAR method (Situation, Task, Action, Result) format to describe each experience.
       - Use strong action verbs, quantify achievements if metrics are available, and prioritize relevance.
       - Avoid repetition of action verbs and overuse of words.
       - Ensure each bullet point aligns with the job description and highlights the candidate's relevant skills and achievements.

    3. **Ethics and Integrity**:
       - Do not invent new accomplishments or metrics.
       - Keep the information truthful and concise.

    4. **Output Format**:
       - Provide the optimized "Experience" section in valid JSON format.
       - The output must only contain the updated "Experience" section and nothing else.

    ### Candidate Profile:
    {json.dumps(candidate_profile, indent=4)}

    ### Job Description:
    {json.dumps(job_description, indent=4)}
    """

    try:
        # Step 2: Query the LLM for optimization
        completion = client.chat.completions.create(
            model="llama3-70b-8192",  # Replace with your model name
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=2048,
            top_p=0.9,
            stream=False,
            stop=None,
        )

        if not (completion.choices and completion.choices[0].message):
            print("No valid response received from the LLM.")
            return

        first_output = completion.choices[0].message.content

        # Save the raw output to a file for reference
        with open("experience_raw.txt", "w") as f:
            f.write(first_output)

        # Step 3: Extract only the JSON object
        second_prompt = f"""
        You are a JSON parser. Extract only the valid JSON object from the following text without any additional explanation.

        ### Input:
        {first_output}

        ### Output:
        Provide only the JSON object.
        """

        second_response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[{"role": "user", "content": second_prompt}],
            temperature=0.3,
            max_tokens=1024,
            top_p=0.9,
            stream=False,
            stop=None,
        )

        if not (second_response.choices and second_response.choices[0].message):
            print("No valid response received for JSON extraction.")
            return

        second_output = second_response.choices[0].message.content

        # Attempt to save the JSON object, retrying up to max_trials
        for attempt in range(max_trials):
            try:
                extracted_json = json.loads(second_output.strip())
                with open("experience.json", "w") as file:
                    json.dump(extracted_json, file, indent=4)
                print("Optimized experience JSON saved to 'experience.json'.")
                return
            except json.JSONDecodeError:
                print(f"Trial {attempt + 1}/{max_trials}: Error parsing JSON. Retrying...")
                time.sleep(1)  # Optional delay before retry

        print("Failed to parse and save JSON after maximum retries.")

    except Exception as e:
        print(f"Error in LLM processing: {e}")


In [45]:
analyze_exp(candidate_exp,job_description)

Optimized experience JSON saved to 'experience.json'.
